# Anatomy of a QualibrationNode

This guide dissects a typical QUAlibrate calibration node, using the `time_of_flight.py` script as a concrete example. We will go through the script section by section, presenting the code first, followed by a detailed explanation of its purpose and functionality within the node's workflow.

All calibration node scripts follow a standardized structure, segmented into distinct sections using `# %% {Section Name}` separators. This consistent structure makes nodes easier to understand, maintain, and integrate. This segmentation also allows developers to run individual cells interactively within a Python kernel (e.g., in VS Code), which is highly beneficial for development and debugging.

The common sections found in most calibration nodes are:
* `Imports`: Handles all necessary library and module imports.
* `Initialisation`: Defines the node, description and parameters.
* `QUA_program`: Defines the core QUA pulse sequence for the experiment.
* `Simulate` (Optional): Simulates the QUA program's waveforms and timing.
* `Execute`: Runs the QUA program on the quantum hardware and fetches results.
* `Load_data` (Optional): Loads data from a previous run instead of executing.
* `Analyse_data`: Processes the raw data and performs fitting or analysis.
* `Plot_data`: Generates plots visualizing the data and analysis results.
* `Save_results`: Persists all parameters, results and figures.

Each of these common sections typically corresponds to a Python function decorated with `@node.run_action`. This decorator effectively encapsulates the logic for that specific step (like creating the QUA program or analysing data) and registers it as an action associated with the node instance, much like a method belongs to a class object. This approach promotes modularity and allows the QUAlibrate framework to manage the execution flow, including conditional skipping of steps.

## Imports Section

In [ ]:
# %% {Imports}
import matplotlib.pyplot as plt

from configuration.configuration_with_lf_fem_and_mw_fem import *

from qm import QuantumMachinesManager
from qm.qua import *

from qualang_tools.results import progress_counter, fetching_tool

from qualibrate import QualibrationNode
from calibration_utils.time_of_flight import (
    Parameters,
    process_raw_data,
    fit_raw_data,
    plot_single_run_with_fit,
    plot_averaged_run_with_fit,
)
from qualibration_libs.runtime import simulate_and_plot

### Explanation: Imports Section

This initial section handles importing all necessary libraries and modules.
- **Standard Libraries:** Imports standart useful libraries, for example `matplotlib.pyplot` for plotting.
- **Configuration file:** Imports the configuration dictionary from the configuration file.
- **QM Libraries:** Imports everything from `qm.qua` (like `program`, `declare`, `measure`, `play`, QUA control flow statements like `for_`, `save`, `stream_processing`, etc.).
- **Qualang Tools:** Imports specific utilities like `progress_counter` (for displaying progress during data acquisition), and `fetching_tool` (for fetching data).
- **QUAlibrate:** Imports `QualibrationNode`, which is the core class for creating calibration nodes.
- **Experiment-Specific Imports:** Imports the `Parameters` class specific to this resonator spectroscopy experiment, along with functions for processing (`process_raw_dataset`), fitting (`fit_raw_data`),  and plotting (`plot_single_run_with_fit`, `plot_averaged_run_with_fit`) from the corresponding `qualibration_utils.time_of_flight` package.
- **Workflow/Helper Imports:** The package `qualibration_libs` contains a series of tools that are generally useful for calibration nodes, as opposed to being useful for a specific node. This includes `simulate_and_plot`, a workflow utility for running simulations.

## Initialisation Section

In [ ]:
# %% {Node initialisation}
description = """
        TIME OF FLIGHT
This sequence involves sending a readout pulse and capturing the raw ADC traces.
The data undergoes post-processing to calibrate three distinct parameters:
    - Time of Flight: This represents the internal processing time and the propagation delay of the readout pulse.
    Its value can be adjusted in the configuration under "time_of_flight".
    This value is utilized to offset the acquisition window relative to when the readout pulse is dispatched.

    - Analog Inputs Offset: Due to minor impedance mismatches, the signals captured by the OPX might exhibit slight offsets.
    These can be rectified in the configuration at: config/controllers/"con1"/analog_inputs, enhancing the demodulation process.

    - Analog Inputs Gain: If a signal is constrained by digitization or if it saturates the ADC,
    the variable gain of the OPX analog input can be modified to fit the signal within the ADC range of +/-0.5V.
    This gain, ranging from -12 dB to 20 dB, can also be adjusted in the configuration at: config/controllers/"con1"/analog_inputs.
"""

node = QualibrationNode[Parameters, None](
    name="time_of_flight",  # Name should be unique
    description=description,  # Describe what the node is doing, which is also reflected in the QUAlibrate GUI
    parameters=Parameters(),  # Node parameters defined under calibration_utils/node_name/parameters
)


# Any parameters that should change for debugging purposes only should go in here
# These parameters are ignored when run through the GUI
@node.run_action(skip_if=node.modes.external)
def custom_param(node: QualibrationNode[Parameters, None]):
    """Allow the user to locally set the node parameters for debugging purposes, or execution in the Python IDE."""
    # You can get type hinting in your IDE by typing node.parameters.
    node.parameters.simulate = False
    node.parameters.resonators = ["q1_resonator", "q2_resonator"]
    node.parameters.multiplexed = True
    node.parameters.num_shots = 10
    node.parameters.depletion_time = 10 * u.us
    pass

### Explanation: Initialisation Section

This section sets up the core `QualibrationNode` object.
- **Description String:** A multi-line string `description` is defined. This provides a human-readable explanation of the node's purpose, prerequisites, and the parameters it intends to update. This description is displayed in the QUAlibrate user interface.
- **Node Instantiation:** An instance of `QualibrationNode` is created.
    - `QualibrationNode[Parameters, None]`: Type hints are used to associate the node with the specific `Parameters` class (imported earlier). This aids IDEs with type checking and autocompletion.
    - `name="time_of_flight"`: Assigns a unique identifier to this node type. This name is crucial for QUAlibrate to track and manage the node.
    - `description=description`: Passes the description string defined above.
    - `parameters=Parameters()`: Passes an *instance* of the imported `Parameters` class. QUAlibrate uses this to manage the node's input parameters (reading defaults, accepting user input via the UI).
- **Custom Parameter Run Action (`custom_param`):**
    - `@node.run_action(skip_if=node.modes.external)`: Defines a function `custom_param` as a run action. The `skip_if=node.modes.external` argument ensures this function *only* runs when the script is executed directly (standalone mode, e.g., in an IDE) and is *skipped* when run via the QUAlibrate UI or as part of a graph.
    - **Purpose:** This action allows developers to temporarily override parameters (like `node.parameters.resonators = ["q1_resonator", "q2_resonator"]`) for local debugging or testing without affecting the default parameters used in automated runs.


## Create QUA Program Section

In [ ]:
# %% {QUA_program}
@node.run_action(skip_if=node.parameters.load_data_id is not None)
def create_qua_program(node: QualibrationNode[Parameters, None]):
    """Create the sweep axes and generate the QUA program from the pulse sequence and the node parameters."""
    # Get the active qubits from the node and organize them by batches
    resonators = node.parameters.resonators
    node.namespace["sweep_axes"] = {
        "resonator": resonators,
    }
    with program() as node.namespace["qua_program"]:
        n = declare(int)  # QUA variable for the averaging loop
        n_st = declare_stream()
        adc_st = [
            declare_stream(adc_trace=True) for _ in range(len(resonators))
        ]  # The stream to store the raw ADC trace

        with for_(n, 0, n < node.parameters.num_shots, n + 1):
            save(n, n_st)
            for i, resonator in enumerate(resonators):
                # Reset the phase of the digital oscillator associated to the resonator element. Needed to average the cosine signal.
                reset_if_phase(resonator)
                # Measure the resonator (send a readout pulse and record the raw ADC trace)
                measure(
                    "readout",
                    resonator,
                    adc_stream=adc_st[i],
                )
                # Wait for the resonator to deplete
                wait(node.parameters.depletion_time * u.ns, resonator)
                if not node.parameters.multiplexed:
                    align()

        with stream_processing():
            n_st.save("n")
            for i, resonator in enumerate(resonators):
                # Will save average:
                adc_st[i].input1().real().average().save(f"adcI{i + 1}")
                adc_st[i].input1().image().average().save(f"adcQ{i + 1}")
                # Will save only last run:
                adc_st[i].input1().real().save(f"adc_single_runI{i + 1}")
                adc_st[i].input1().image().save(f"adc_single_runQ{i + 1}")

### Explanation: Create_QUA_program Section

This run action defines the core QUA pulse sequence for time of flight calibration. Its main goal is to generate a QUA program that captures the raw ADC data from the resonator, in order to calibrated the time of flight, analog inpus offsets and the analog input gain. 

- **Preparations:** The action first defines relevant variables for the QUA probram. 
  Relevant sweeps are collected into `node.namespace["sweep_axes"]`, to be used within the QUA program.
- **Program Definition** It then enters the `with program()` context to define the QUA sequence. Inside, it declares necessary QUA variables and streams, often using helper methods from the loaded `Quam` object (`node.machine`) for consistency.
- **Nested Loops:** The core logic involves nested loops:
    1.  An averaging loop (`with for_(n,...)`) repeats the measurement `n_avg` times.
    2.  An inner loop (`for i, resonator in enumerate(resonators):`) that runs over the resonators declared in `node.parameters.resonators`. 
- **Measurement Sequence:** Inside the inner loop, for each resonator:
    1.  The resonator's phase is updated (`reset_if_phase(...)`).
    2.  A measurement pulse is played, and the the raw ADC trace is saved to the stream processing block (`measure("readout", ...)`).
    3.  A wait time allows the resonator to relax (`wait(...)`).
- **Stream Processing:** After the loops complete, a `with stream_processing()` block defines how the QOP should process the raw data streams. Here, it saves the averaged raw ADC data over all `n_avg` iterations and the last measurement.
- **Output:** The generated QUA program is stored in `node.namespace["qua_program"]`.

## Simulate Section

In [ ]:
@node.run_action(skip_if=node.parameters.load_data_id is not None or not node.parameters.simulate)
def simulate_qua_program(node: QualibrationNode[Parameters, None]):
    """Connect to the QOP and simulate the QUA program"""
    # Connect to the QOP
    qmm = QuantumMachinesManager(host=qop_ip, cluster_name=cluster_name)
    # Simulate the QUA program, generate the waveform report and plot the simulated samples
    samples, fig, wf_report = simulate_and_plot(qmm, config, node.namespace["qua_program"], node.parameters)
    # Store the figure, waveform report and simulated samples
    node.results["simulation"] = {"figure": fig, "wf_report": wf_report, "samples": samples}

### Explanation: Simulate Section

This optional run action allows simulating the QUA program before running it on actual hardware.
- **Decorator:** `@node.run_action(skip_if=node.parameters.load_data_id is not None or not node.parameters.simulate)` ensures this action only runs if data isn't being loaded (`load_data_id` is None) AND the `simulate` parameter is set to `True`.
- **Connection:**
    - `qmm = QuantumMachinesManager()`: Establishes a connection to the Quantum Machines Manager (QMM).
- **Simulation Execution:**
    - `samples, fig, wf_report = simulate_and_plot(...)`: Calls a workflow utility function (`simulate_and_plot` from `quam_experiments.workflow`) which likely wraps the standard `qmm.simulate` call. It passes the QMM object, the generated config, the QUA program (stored in `node.namespace["qua_program"]`), and the node parameters. This function typically returns the simulated waveforms (`samples`), a figure visualizing them (`fig`), and potentially a waveform report (`wf_report`).
- **Result Storage:**
    - `node.results["simulation"] = {"figure": fig, "wf_report": wf_report, "samples": samples}`: Stores the outputs of the simulation (figure, report, samples) in the `node.results` dictionary under the key "simulation".

## Execute Section

In [ ]:
@node.run_action(skip_if=node.parameters.load_data_id is not None or node.parameters.simulate)
def execute_qua_program(node: QualibrationNode[Parameters, Quam]):
    """Connect to the QOP, execute the QUA program and fetch the raw data and store it in a xarray dataset called "ds_raw"."""
    # Connect to the QOP
    qmm = QuantumMachinesManager(host=qop_ip, cluster_name=cluster_name)
    # Execute the QUA program only if the quantum machine is available (this is to avoid interrupting running jobs).
    qm = qmm.open_qm(config)
    # The job is stored in the node namespace to be reused in the fetching_data run_action
    node.namespace["job"] = job = qm.execute(node.namespace["qua_program"])
    # Names and values mapping
    keys = []

    for i in range(1, len(node.parameters.resonators) + 1):
        keys.extend([f"adcI{i}", f"adcQ{i}", f"adc_single_runI{i}", f"adc_single_runQ{i}"])
    data_fetcher = fetching_tool(job, data_list=keys, mode="wait_for_all")
    values = data_fetcher.fetch_all()
    # Display the execution report to expose possible runtime errors
    node.log(job.execution_report())
    # Register the raw dataset
    node.results["raw_data"] = {}
    for key, value in zip(keys, values):
        node.results["raw_data"][key] = value

### Explanation: Execute Section

This run action executes the QUA program on the actual hardware (QOP) and fetches the results.
- **Decorator:** `@node.run_action(skip_if=node.parameters.load_data_id is not None or node.parameters.simulate)` ensures this action only runs if data isn't being loaded AND simulation is not enabled.
- **Connection & Config:** It connects to the QMM (`QuantumMachinesManager`) and open the quantum machine (`qmm.open_qm()`).
- **Job Execution:**
    - `node.namespace["job"] = job = qm.execute(node.namespace["qua_program"])`: Executes the QUA program stored in the namespace and stores the returned `job` object in the namespace as well.
- **Data Fetching & Progress:**
    - `fetching_tool(job, data_list=keys, mode="wait_for_all")`: This tool handles fetching data from the QOP streams.
- **Execution Report:** `node.log(job.execution_report())`: After the job finishes, prints a report containing information about the execution, including any potential runtime errors.
- **Result Storage:** `node.results[]`: Stores the final fetched dataset.

## Load Data Section

In [ ]:
@node.run_action(skip_if=node.parameters.load_data_id is None)
def load_data(node: QualibrationNode[Parameters, None]):
    """Load a previously acquired dataset."""
    load_data_id = node.parameters.load_data_id
    # Load the specified dataset
    node.load_from_id(node.parameters.load_data_id)
    node.parameters.load_data_id = load_data_id

### Explanation: Load_data Section

This run action provides an alternative to executing the experiment; it loads data from a previous run identified by an ID.
- **Decorator:** `@node.run_action(skip_if=node.parameters.load_data_id is None)` ensures this action only runs if the `load_data_id` parameter *is* set (i.e., not None).
- **Loading:**
    - `load_data_id = node.parameters.load_data_id`: Stores the ID locally.
    - `node.load_from_id(node.parameters.load_data_id)`: Calls the built-in `QualibrationNode` method to load the parameters and results associated with the specified run ID from the data storage location. This populates `node.parameters` and `node.results` with the loaded data.
    - `node.parameters.load_data_id = load_data_id`: Resets the parameter in the current node instance (loading might overwrite it).

## Analyse Data Section

In [ ]:
@node.run_action(skip_if=node.parameters.simulate)
def analyse_data(node: QualibrationNode[Parameters, None]):
    """Analyse the raw data and store the fitted data in node.results."""
    num_resonators = len(node.parameters.resonators)
    node.results["processed_data"] = process_raw_data(node.results["raw_data"])
    node.results["fitted_data"] = fit_raw_data(node.results["processed_data"], num_resonators)

### Explanation: Analyse_data Section

This run action performs post-processing and fitting on the acquired (or loaded) raw data.
- **Decorator:** `@node.run_action(skip_if=node.parameters.simulate)` ensures this action is skipped if the node is only simulating the program.
- **Data Processing:**
    - `process_raw_dataset(node.results)`: Calls an experiment-specific function (`process_raw_dataset`) to perform initial processing on the raw data stored in `node.results`. This might involve calculating magnitude and phase from I/Q, unit conversions, or background subtraction. The processed dataset is then stored in node.results["processed_data"].
- **Fitting:**
    - `fit_raw_data(node.results, node)`: Calls another experiment-specific function (`fit_raw_data`) to fit a model to the processed data. Then, the fitted data is dtored in node.results["fitted_data"].

## Plot Data Section

In [ ]:
@node.run_action(skip_if=node.parameters.simulate)
def plot_data(node: QualibrationNode[Parameters, None]):
    """Plot the raw and fitted data."""
    num_resonators = len(node.parameters.resonators)
    fig_single_run_fit = plot_single_run_with_fit(node.results, num_resonators)
    fig_averaged_run_fit = plot_averaged_run_with_fit(node.results, num_resonators)
    node.results["figures"] = {
        "single_run": fig_single_run_fit,
        "averaged_run": fig_averaged_run_fit,
    }

### Explanation: Plot_data Section

This run action generates plots based on the processed and fitted data.
- **Decorator:** `@node.run_action(skip_if=node.parameters.simulate)` skips plotting during simulation.
- **Plot Generation:**
    - Calls experiment-specific plotting functions (e.g., `plot_single_run_with_fit`, `plot_averaged_run_with_fit`) imported earlier. These functions take the relevant data (`node.results`) and the number of resonators as input and return Matplotlib figure objects.
- **Display Plot:** `plt.show()`: Displays the generated plots interactively when the script is run standalone. This line might be commented out for fully automated runs.
- **Store Figures:**
    - `node.results["figures"] = {"single_run": fig_single_run_fit,
        "averaged_run": fig_averaged_run_fit,}`: Stores the generated Matplotlib figure objects in the `node.results` dictionary under the key "figures". QUAlibrate automatically saves these figures when `node.save()` is called, making them viewable in the UI.

## Save Results Section

In [ ]:
@node.run_action()
def save_results(node: QualibrationNode[Parameters, None]):
    node.save()

### Explanation: Save_results Section

This final run action is responsible for persisting all the relevant information gathered during the node's execution.
- **Decorator:** `@node.run_action()`: Typically has no `skip_if` condition, ensuring that saving always occurs (unless the node fails catastrophically earlier).
- **Saving Operation:** `node.save()`: Calls the core `save` method of the `QualibrationNode` instance. This method performs several actions:
    - Saves the input parameters (`node.parameters`) used for this specific run.
    - Saves the contents of the `node.results` dictionary, including any data and Matplotlib figures.
- **Data Location:** The data is saved in a structured directory hierarchy managed by QUAlibrate using the timestamp, unique run id, and node name, allowing for easy retrieval and comparison of results across different runs.